# Penalty kicks with types

In [42]:
import numpy as np 
import pandas as pd 
import nashpy
import bimatrix

In [3]:
def print_payoffs(U1, U2, A1=None, A2=None, player1=None, player2=None): 
    na1,na2 = U1.shape
    tab = pd.DataFrame([[(U1[r,c],U2[r,c]) for c in range(na2)] for r in range(na1)])

    # nice headings 
    if A1 is not None: 
        tab.index = A1
    if A2 is not None:
        tab.columns = A2
    if player1 is not None:
        tab.index.name = player1
    if player2 is not None:
        tab.columns.name = player2
    
    return tab 

# Input data

In [4]:
# names of the actions 
A = ['L','C','R']

In [5]:
# frequency of kicks by (kicker, goalie) actions
F = np.array([
    [117, 48, 95],
    [4, 3, 4], 
    [85, 28, 75]
])
tab = pd.DataFrame(F, index=A, columns=A)
tab.index.name = 'Goalie'
tab.columns.name = 'Kicker' 

# action frequencies in the data 
a_kicker = tab.sum(0) / tab.sum().sum()
a_goalie = tab.sum(1) / tab.sum().sum()

print(f'--- Action frequencies ---')
pd.DataFrame({'Goalie': a_goalie, 'Kicker': a_kicker}, index=A).round(2)

--- Action frequencies ---


,Goalie,Kicker
L,0.57,0.45
C,0.02,0.17
R,0.41,0.38


In [6]:
# Pr(save|kicker is left-legged)
UL = np.array([
    [0.418, 0.188, 0.055],
    [0.   , 1.   , 0.   ],
    [0.109, 0.107, 0.51 ]
])
# Pr(save|kicker is right-legged)
UR = np.array([
    [0.318, 0.188, 0.155],
    [0.   , 1.   , 0.   ],
    [0.009, 0.107, 0.61 ]
])

In [7]:
saves_leftie = pd.DataFrame(UL, index=A, columns=A)
saves_leftie.index.name = 'Goalie'
saves_leftie.columns.name = 'Kicker'
saves_leftie

Kicker,L,C,R
Goalie,,,
L,0.418,0.188,0.055
C,0.000,1.000,0.000
R,0.109,0.107,0.510


In [8]:
saves_rightie = pd.DataFrame(UR, index=A, columns=A)
saves_rightie.index.name = 'Goalie'
saves_rightie.columns.name = 'Kicker'
saves_rightie

Kicker,L,C,R
Goalie,,,
L,0.318,0.188,0.155
C,0.000,1.000,0.000
R,0.009,0.107,0.610


Set up the sub-game where the kicker is left-legged and this is known by both players as a normal form game. 

In [9]:
t = print_payoffs(UL, -UL, A, A, 'Goalie', 'Kicker')
t

Kicker,L,C,R
Goalie,,,
L,"(0.418, -0.418)","(0.188, -0.188)","(0.055, -0.055)"
C,"(0.0, -0.0)","(1.0, -1.0)","(0.0, -0.0)"
R,"(0.109, -0.109)","(0.107, -0.107)","(0.51, -0.51)"


## 1.1

In [10]:
game_known_left = nashpy.Game(UL, -UL)
equilibria_known_left = list(game_known_left.support_enumeration())
print(f'Goalie strategy: {equilibria_known_left[0][0]} \nKicker strategy: {equilibria_known_left[0][1]}')

Goalie strategy: [0.46793534 0.10847231 0.42359234] 
Kicker strategy: [0.42593202 0.24176854 0.33229944]


In [11]:
game_known_right = nashpy.Game(UR, -UR)
equilibria_known_right = list(game_known_right.support_enumeration())
print(f'Goalie strategy: {equilibria_known_right[0][0]} \nKicker strategy: {equilibria_known_right[0][1]}')

Goalie strategy: [0.72746551 0.07523519 0.1972993 ] 
Kicker strategy: [0.43200679 0.23310973 0.33488348]


# 1.2

In [43]:
print(f'--- Hvis angriberen er vesntrebenet og kun selv ved det ---')
bimatrix.print_payoffs([0.11*UL+0.89*UR, 1-UL], [A, A],2)


--- Hvis angriberen er vesntrebenet og kun selv ved det ---


,L,C,R
L,"(0.33, 0.58)","(0.19, 0.81)","(0.14, 0.94)"
C,"(0.0, 1.0)","(1.0, 0.0)","(0.0, 1.0)"
R,"(0.02, 0.89)","(0.11, 0.89)","(0.6, 0.49)"


In [44]:
print(f'--- Hvis angriberen er vesntrebenet og kun selv ved det ---')
bimatrix.print_payoffs([0.11*UL+0.89*UR, 1-UR], [A, A],2)

--- Hvis angriberen er vesntrebenet og kun selv ved det ---


,L,C,R
L,"(0.33, 0.68)","(0.19, 0.81)","(0.14, 0.84)"
C,"(0.0, 1.0)","(1.0, 0.0)","(0.0, 1.0)"
R,"(0.02, 0.99)","(0.11, 0.89)","(0.6, 0.39)"


In [45]:
p = 0.11
U_goalie = p * UL + (1 - p) * UR

In [46]:
def compute_full_matrix(U1, U2, p, action_names=None): 
    '''
        Assumes that only player 2's type varies 
        (this means that player 1 has one action per row in U1, 
         while 2 has nA2**2 (one choice per type))
        Both players have one utility matrix for each realization 
        of player 2's type. 
         
        INPUTS: 
            U1: list of 2 payoff matrices for player 1 (row player)
            U2: list of 2 payoff matrices for player 2 (column player)
            p: (scalar) Probability that player 2 is the first type 
            action_names: [optional] 2-list of names of actions (nA1 and nA2 long)
        OUTPUTS: 
            t1, t2: wide-form payoff matrices suitable for finding the NE 
            A1, A2: names of actions 
    '''
    assert len(U1) == 2
    assert len(U2) == 2 
    assert np.isscalar(p)
    nA1, nA2 = U1[0].shape
    
    t1 = np.empty((nA1, nA2*nA2))
    t2 = np.empty((nA1, nA2*nA2))
    
    # player 1 chooses an action without knowing what type 2 is 
    for ia1 in range(nA1): 
        i_col = 0 
        
        # player 2 chooses an action conditional on observing her type 
        for a2_1 in range(nA2): 
            for a2_2 in range(nA2): 
                t1[ia1,i_col] = p * U1[0][ia1,a2_1] + (1.-p) * U1[1][ia1,a2_2]
                t2[ia1,i_col] = p * U2[0][ia1,a2_1] + (1.-p) * U2[1][ia1,a2_2]
                
                i_col += 1
                
    if action_names is None: 
        A1 = [f'{i}' for i in range(nA1)]
        A2 = [f'{a}{b}' for a in range(nA2) for b in range(nA2)]
    else: 
        assert len(action_names) == 2 
        A1 = action_names[0]
        assert len(A1) == nA1, f'Incorrect # of action names'
        a2 = action_names[1]
        assert len(a2) == nA2, f'Incorrect # of action names'
        
        A2 = [f'{a}{b}' for a in a2 for b in a2]
        
    return t1, t2, A1, A2

In [48]:
t1, t2, A1, A2 = compute_full_matrix([U_goalie, U_goalie], [1-UL, 1-UR], 0.11, [A, A])

In [49]:
bimatrix.print_payoffs([t1, t2], [A1,  A2], 3)

,LL,LC,LR,CL,CC,CR,RL,RC,RR
L,"(0.329, 0.671)","(0.204, 0.787)","(0.164, 0.816)","(0.313, 0.696)","(0.188, 0.812)","(0.149, 0.841)","(0.309, 0.711)","(0.183, 0.827)","(0.144, 0.856)"
C,"(0.0, 1.0)","(0.89, 0.11)","(0.0, 1.0)","(0.11, 0.89)","(1.0, 0.0)","(0.11, 0.89)","(0.0, 1.0)","(0.89, 0.11)","(0.0, 1.0)"
R,"(0.02, 0.98)","(0.097, 0.893)","(0.535, 0.445)","(0.03, 0.98)","(0.107, 0.893)","(0.545, 0.445)","(0.084, 0.936)","(0.161, 0.849)","(0.599, 0.401)"


In [50]:
A_, T_ = IESDS([A1, A2], [t1, t2], DOPRINT=True)

In [51]:
print_payoffs(T_, A_, 3)

,LL,LC,LR,CL,CC,CR,RL,RC,RR
L,"(0.329, 0.671)","(0.204, 0.787)","(0.164, 0.816)","(0.313, 0.696)","(0.188, 0.812)","(0.149, 0.841)","(0.309, 0.711)","(0.183, 0.827)","(0.144, 0.856)"
C,"(0.0, 1.0)","(0.89, 0.11)","(0.0, 1.0)","(0.11, 0.89)","(1.0, 0.0)","(0.11, 0.89)","(0.0, 1.0)","(0.89, 0.11)","(0.0, 1.0)"
R,"(0.02, 0.98)","(0.097, 0.893)","(0.535, 0.445)","(0.03, 0.98)","(0.107, 0.893)","(0.545, 0.445)","(0.084, 0.936)","(0.161, 0.849)","(0.599, 0.401)"


In [52]:
eqs = list(nashpy.Game(T_[0], T_[1]).support_enumeration())
print(f'Found {len(eqs)} equilibria')
for i,eq in enumerate(eqs): 
    print(f'{i+1}: s1 = {eq[0]}, s2 = {eq[1]}')

print(f'Goalie strategy: {eqs[0][0]} \nKicker strategy: {eqs[0][1]}')

Found 1 equilibria
1: s1 = [0.72746551 0.07523519 0.1972993 ], s2 = [0.         0.         0.         0.         0.         0.
 0.48462526 0.2630266  0.25234814]
Goalie strategy: [0.72746551 0.07523519 0.1972993 ] 
Kicker strategy: [0.         0.         0.         0.         0.         0.
 0.48462526 0.2630266  0.25234814]


# 1.3

In [53]:
p=0.89
U_goalie = p * UL + (1 - p) * UR

In [54]:
t1, t2, A1, A2 = compute_full_matrix([U_goalie, U_goalie], [1-UL, 1-UR], 0.89, [A, A])

In [55]:
bimatrix.print_payoffs([t1, t2], [A1,  A2], 3)

,LL,LC,LR,CL,CC,CR,RL,RC,RR
L,"(0.407, 0.593)","(0.383, 0.607)","(0.369, 0.611)","(0.212, 0.798)","(0.188, 0.812)","(0.175, 0.816)","(0.104, 0.916)","(0.079, 0.93)","(0.066, 0.934)"
C,"(0.0, 1.0)","(0.11, 0.89)","(0.0, 1.0)","(0.89, 0.11)","(1.0, 0.0)","(0.89, 0.11)","(0.0, 1.0)","(0.11, 0.89)","(0.0, 1.0)"
R,"(0.098, 0.902)","(0.099, 0.891)","(0.145, 0.836)","(0.106, 0.904)","(0.107, 0.893)","(0.153, 0.838)","(0.474, 0.545)","(0.475, 0.534)","(0.521, 0.479)"


In [56]:
A_, T_ = IESDS([A1, A2], [t1, t2], DOPRINT=True)

In [57]:
print_payoffs(T_, A_, 3)

,LL,LC,LR,CL,CC,CR,RL,RC,RR
L,"(0.407, 0.593)","(0.383, 0.607)","(0.369, 0.611)","(0.212, 0.798)","(0.188, 0.812)","(0.175, 0.816)","(0.104, 0.916)","(0.079, 0.93)","(0.066, 0.934)"
C,"(0.0, 1.0)","(0.11, 0.89)","(0.0, 1.0)","(0.89, 0.11)","(1.0, 0.0)","(0.89, 0.11)","(0.0, 1.0)","(0.11, 0.89)","(0.0, 1.0)"
R,"(0.098, 0.902)","(0.099, 0.891)","(0.145, 0.836)","(0.106, 0.904)","(0.107, 0.893)","(0.153, 0.838)","(0.474, 0.545)","(0.475, 0.534)","(0.521, 0.479)"


In [58]:
eqs = list(nashpy.Game(T_[0], T_[1]).support_enumeration())
print(f'Found {len(eqs)} equilibria')
for i,eq in enumerate(eqs): 
    print(f'{i+1}: s1 = {eq[0]}, s2 = {eq[1]}')

print(f'Goalie strategy: {eqs[0][0]} \nKicker strategy: {eqs[0][1]}')

Found 1 equilibria
1: s1 = [0.46793534 0.10847231 0.42359234], s2 = [0.3557065  0.         0.         0.27061422 0.         0.
 0.37367928 0.         0.        ]
Goalie strategy: [0.46793534 0.10847231 0.42359234] 
Kicker strategy: [0.3557065  0.         0.         0.27061422 0.         0.
 0.37367928 0.         0.        ]
